<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Auto Generated Agent: Task Solving with Langchain Provided Tools as Functions

In [1]:
import autogen

# config_list = autogen.config_list_from_json(
#     "./OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-4-1106-preview"],
#     },
# )

## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the default system prompt of `AssistantAgent`, we allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `AssistantAgent`. Then we register the functions in `UserProxyAgent`.


In [2]:
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import format_tool_to_openai_function

tsearch = TavilySearchResults()

In [4]:
format_tool_to_openai_function(tsearch)

{'name': 'tavily_search_results_json',
 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
 'parameters': {'title': 'TavilyInput',
  'description': 'Input for the Tavily tool.',
  'type': 'object',
  'properties': {'query': {'title': 'Query',
    'description': 'search query to look up',
    'type': 'string'}},
  'required': ['query']}}

In [14]:
# Construct the llm_config
llm_config = {
    # Generate functions config for the Tool
    "functions": [format_tool_to_openai_function(tsearch)],
    "config_list": config_list,  # Assuming you have this defined elsewhere
    "timeout": 120,
}

In [15]:
llm_config

{'functions': [{'name': 'tavily_search_results_json',
   'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
   'parameters': {'title': 'TavilyInput',
    'description': 'Input for the Tavily tool.',
    'type': 'object',
    'properties': {'query': {'title': 'Query',
      'description': 'search query to look up',
      'type': 'string'}},
    'required': ['query']}}],
 'config_list': [{'model': 'ollama/dolphin-mistral',
   'api_key': 'NULL',
   'base_url': 'http://localhost:8000'}],
 'timeout': 120}

In [16]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding", "use_docker": False},
)

# Register the tool and start the conversation
user_proxy.register_function(function_map={tsearch.name: tsearch._run})

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="Use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy.initiate_chat(
    chatbot,
    message="Gather information about an AI vision model called 'LLAVA', then extract a list of the most important information.",
    llm_config=llm_config,
)

user_proxy (to chatbot):

Gather information about LLAVA, then extract a list of the most important information.

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function Call: tavily_search_results_json *****
Arguments: 
{"query":"LLAVA"}
***************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION tavily_search_results_json...
user_proxy (to chatbot):

***** Response from calling function "tavily_search_results_json" *****
[{'url': 'https://apnews.com/article/iceland-volcano-grindavik-eruption-8202c1c858baabb18251a4fa032bcac9', 'content': 'Monday, Jan. 15, 2024. Iceland’s president says the country is battling “tremendous forces of nature” after molten lava  Monday, Jan. 15, 2024. Iceland’s president says the country is battling “tremendous forces of nature” after molten lava  Lava flows from the F

## OK. This time with local LLM.

In [11]:
config_list = autogen.config_list_from_json("./ORCA_CONFIG_LIST")

# Construct the llm_config
llm_config = {
    # Generate functions config for the Tool
    "functions": [format_tool_to_openai_function(tsearch)],
    "config_list": config_list,  # Assuming you have this defined elsewhere
    "timeout": 120,
}

In [12]:
llm_config

{'functions': [{'name': 'tavily_search_results_json',
   'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
   'parameters': {'title': 'TavilyInput',
    'description': 'Input for the Tavily tool.',
    'type': 'object',
    'properties': {'query': {'title': 'Query',
      'description': 'search query to look up',
      'type': 'string'}},
    'required': ['query']}}],
 'config_list': [{'model': 'dolphin-mistral',
   'api_key': 'NULL',
   'base_url': 'http://localhost:8000'}],
 'timeout': 120}

In [13]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding", "use_docker": False},
    llm_config=llm_config,
)

# Register the tool and start the conversation
user_proxy.register_function(function_map={tsearch.name: tsearch._run})

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="When gatering information, only use the function 'tavily_search_results_json'. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy.initiate_chat(
    chatbot,
    message="Gather information from the internet about an AI vision model called 'LLAVA', then extract a list of the most important information.",
    llm_config=llm_config,
)

user_proxy (to chatbot):

Gather information from the internet about an AI vision model called 'LLAVA', then extract a list of the most important information.

--------------------------------------------------------------------------------


chatbot (to user_proxy):

***** Suggested tool Call (call_c3a8f875-bbb4-41e9-9c54-64a51b769091):  *****
Arguments: 
{"search_query": "LLAVA AI vision model",
 "results": [
   {
     "title": "What is LLAVA?",
     "description": "LLAVA (Large Language-Action-Visual Agents) is a transformer-based AI vision model that combines natural language understanding, computer vision, and reasoning abilities. It was developed by researchers at the University of California, Berkeley and the University of Illinois Urbana-Champaign."
   },
   {
     "title": "LLAVA's Capabilities",
     "description": "LLAVA can interpret images and text simultaneously, making it capable of understanding the relationship between them. It also possesses reasoning abilities to make inferences about the visual world, enabling it to provide more accurate and informative responses."
   },
   {
     "title": "LLAVA's Architecture",
     "description": "The LLAVA architecture comprises a transformer-based language model tha

APIConnectionError: Connection error.

In [14]:
tsearch._run

<bound method TavilySearchResults._run of TavilySearchResults()>

In [ ]:
tsearch._run